# Import all the important modules and frameworks

In [ ]:
import nltk
import pandas as pd
from wordcloud import WordCloud
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Load Data

In [ ]:
data=pd.read_csv('data.csv',encoding='utf-8',low_memory=False)
data.info()

# Data Preprocessing

In [ ]:
# Remove all the useless columns
data1=data[['id','reviews.doRecommend','reviews.rating','reviews.text','reviews.title','reviews.username']]

In [ ]:
#fill Null Vallues
data1['reviews.title'].fillna(value=data1['reviews.text'],inplace=True)


In [ ]:
data1['reviews.username'].fillna(value=' ',inplace=True)


In [ ]:
data1['reviews.text'].fillna(value=data1['reviews.title'],inplace=True)

In [ ]:
#check null values
index=data1[data1['reviews.doRecommend'].isnull()][data1['reviews.rating'].isnull()].index
index

In [ ]:
# drop the null value rows
data1.drop(index=index,inplace=True,)

In [ ]:
data1.reset_index(inplace=True)

In [ ]:
#Fill the null values
for i in range(len(data1)):
    if data1.loc[i,'reviews.doRecommend']==True:
        data1.loc[i,'doRecommend']=1
    elif data1.loc[i,'reviews.doRecommend']==False:
        data1.loc[i,'doRecommend']=0
    else:
        if data1.loc[i,'reviews.rating']>=4.0:
            data1.loc[i,'doRecommend']=1
        elif data1.loc[i,'reviews.rating']<=2.0:
            data1.loc[i,'doRecommend']=0
        else:
            data1.loc[i,'doRecommend']=0.5

In [ ]:
data1.drop(columns=['reviews.doRecommend','index'],inplace=True)

In [ ]:
data1.isnull().sum()

In [ ]:
data1=data1.sample(frac=1,random_state=88,ignore_index=True)

In [ ]:
data1["final_text"] = data1['reviews.text'].astype(str) +" "+ data1["reviews.title"].astype(str)+" "+ data1["reviews.username"].astype(str)
data1.drop(columns=['reviews.text','reviews.title','reviews.username'],inplace=True)

In [ ]:
def preprocess(tweet):
    # Removing special characters and digits
    letters_only = re.sub("[^a-zA-Z]", " ",tweet)

    # change sentence to lower case
    letters_only = letters_only.lower()

    # tokenize into words
    words = letters_only.split()

    # remove stop words
    words = [w for w in words if not w in stopwords.words("english")]

    # Stemming
    lemma = WordNetLemmatizer()
    words=[lemma.lemmatize(word) for word in words]

    clean_sentence = " ".join(words)

    return clean_sentence


In [ ]:
data1['final_text']=data1['final_text'].apply(preprocess)

In [ ]:
data1.drop(columns='id',inplace=True)

In [ ]:
data1=data1[['final_text','reviews.rating','doRecommend']]

In [ ]:
data1.drop(index=data1[data1['doRecommend']==0.5].index,inplace=True)

In [ ]:
#check for class imbalance
data1['doRecommend'].value_counts()

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

def mapping(text):
    return ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])

In [ ]:
data1['final_text']=data1['final_text'].apply(mapping)

In [ ]:
word2count={}
for data in data1.final_text:
    words=nltk.word_tokenize(data)
    for word in words:
        if word not in word2count.keys():
            word2count[word]=1
        else:
            word2count[word]+=1
            

In [ ]:
import heapq
freq_words = heapq.nlargest(50, word2count, key=word2count.get)
print(freq_words)

In [ ]:
import numpy as np
X = []
for data in data1.final_text:
    vector = []
    for word in freq_words:
        if word in nltk.word_tokenize(data):
            vector.append(1)
        else:
            vector.append(0)
    X.append(vector)
X = np.asarray(X)

In [ ]:
d=pd.DataFrame(X)
d['rating']=data1['reviews.rating']
d['doRecommend']=data1['doRecommend']
d

In [ ]:
d['rating'].fillna(value=3.0,inplace=True)
d['doRecommend'].fillna(value=np.random.randint(0,2),inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(d.iloc[:,:-2],d['doRecommend'],test_size=0.2,shuffle=True,random_state=88)

In [ ]:
from sklearn.linear_model import LogisticRegression
params={'max_iter':[150,250,350,450,550]}
lr=LogisticRegression()
model=GridSearchCV(estimator=lr,param_grid=params,cv=3,scoring='accuracy')
model.fit(X_train,y_train)

In [ ]:
print(model.best_score_)
print(model.best_estimator_)
print(model.best_params_)

final_model=model.best_estimator_
y_predict=final_model.predict(X_test)
print("score is:{}".format( (y_test==y_predict).sum()/len(y_test) ))

In [ ]:
data1

In [ ]:
data1.to_csv("Preprocessed_data.csv")

In [ ]:
data1=pd.read_csv("Preprocessed_data.csv",usecols=['final_text','reviews.rating','doRecommend'])
data1

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import pandas as pd
# we are using dataset of sentence stored in 'dataset' variable.

def create_document_term_matrix(message_list,vectorizer):
    doc_term_matrix=vectorizer.fit_transform(message_list)
    return pd.DataFrame(doc_term_matrix.toarray(),columns=vectorizer.get_feature_names())

In [ ]:
cv=CountVectorizer()
tfidf=TfidfVectorizer()

In [ ]:
cv_vector=create_document_term_matrix(data1['final_text'],cv)
cv_vector['reviews.rating']=data1['reviews.rating']
cv_vector['doRecommend']=data1['doRecommend']

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import numpy as np
X=cv_vector.iloc[:,:50].astype(np.int8)
Y=pd.DataFrame(cv_vector.iloc[:,-1].astype(np.int8))

In [ ]:
from sklearn.linear_model import LogisticRegression
params={'max_iter':[150,250,350,450,550]}
lr=LogisticRegression()
model=GridSearchCV(estimator=lr,param_grid=params,cv=3,scoring='accuracy')
model.fit(X,Y)

In [ ]:
model.best_params_

In [ ]:
model.best_score_

#  Thank you